# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
csv_file = os.path.join('..', 'WeatherPy', 'Output', 'city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [ ]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df['Hotel Name'] = ""
hotel_df

In [ ]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

# Display figure
